# Dynamic Equations

Their code allows us to declare that a variable has a subscript like $t+1$. For example, $x_{t+1}$ would be represented by $x(+1)$

Frequently we want to define something like:

$$
x_t = \rho x_{t-1} + u_t
$$

Here is the most natural way to write it:

In [ ]:
def bad_dynamic(x,rho,u):
    return x(+1) = rho*x + u

This is _not_ a valid code! The right way to write it is to work with the residuals from the equation! 

In [ ]:
def good_dynamic(x,rho,u):
    return res = x(+1) - rho*x - u

# Steady State

Here is one example in their code of a block of code that calculates the steady state in the One Asset HANK example:

In [ ]:
calibration = {'eis': 1, 'frisch': 1, 'rho_e': 0.966, 'sd_e': 0.5, 'nE': 7,
               'amin': 0.0, 'amax': 150, 'nA': 500, 'Y': 1.0, 'Z': 1.0, 'pi': 0.0,
               'mu': 1.2, 'kappa': 0.1, 'rstar': 0.005, 'phi': 1.5, 'B': 5.6}

unknowns_ss = {'beta': 0.986, 'vphi': 0.8}
targets_ss = {'asset_mkt': 0, 'labor_mkt': 0}

ss0 = hank_ss.solve_steady_state(calibration, unknowns_ss, targets_ss, solver="hybr")

print(f"Asset market clearing: {ss0['asset_mkt']: 0.2e}")
print(f"Labor market clearing: {ss0['labor_mkt']: 0.2e}")
print(f"Goods market clearing (untargeted): {ss0['goods_mkt']: 0.2e}")

There are two important things to keep in mind:

## `unknown_ss`

I will start with an example will make it simple: consider the asset market. The amount of assets the household wants to hold depends on the preferences of the household and the real interest rate. In the setady state, the real interest rates depend on the intertemporal discount of the household (think of the usual neoclassical model). We have fixed, in the `calibration` block, the value of the interest rate in the steady state (`rstar`). In order to reach this value, we need to get the exact value of $\beta$ that gives, on steady state, a real interest rate of 0.005. That is the function of `unknown_ss`: it will calibrate the parameter to make the values of quantities in equilibrium as we defined in the `calibration` block. The value that is attributed in the `unknown_ss` is an approximated value to start the algorithm (I am guessing).

We could have set `beta` in the `calibration` block and asked in the `unknown_ss` block what was the real interest rate in the steady state.

Just to drive the point home: $\varphi$ is one of the parameters that goven the desutility of labor. Since the production function is $Y = ZN$, and we defined the values of $N$, $Y$, $Z$ in the steady state (all equal to one), we have to guarantee that, in the steady state, the household will supply $N=1$. In order to guarantee this, we have to select the right value of $\varphi$

Once declared in the `unknown_ss`block, the algorith will automatically find the right values of the variables that create the steady state we want

## `target_ss`

The one asset HANK has three markets: asset market, labor market and good markets. The code only target _two_ markets out of three. If you have $n$ markest, **only** target $n-1$ markets. Otherwise, the steady state solver will always return an error that it was not able to find the steady state.  